# Block Averaging Example


In this example, the block averaging approach to computing error bars is considered.
First, please read the original method: https://doi.org/10.1063/1.457480.
Then, also consider the discussion on block averages in Appendix D of Frenkel and Smit.
Then, we'll reproduce Frenkel and Smith's case study 4 (page 98-100 and Figure 4.4).

In [1]:
import sys
import unittest
import feasst as fst

class TestMonteCarlo4Block(unittest.TestCase):
    def test_srsw(self, num_particles=108, density=0.8442, temperature=1.5, steps_per=1e5):
        monte_carlo = fst.MonteCarlo()
        # monte_carlo.set(fst.MakeRandomMT19937(fst.args({"seed": "1234"})))
        monte_carlo.add(fst.MakeConfiguration(fst.args({
            "cubic_box_length": str((num_particles/density)**(1./3.)),
            "particle_type0": fst.install_dir() + "/forcefield/lj.fstprt",
            "cutoff0": "2.5",
        })))
        monte_carlo.add(fst.MakePotential(fst.MakeLennardJones()))
        monte_carlo.add(fst.MakePotential(fst.MakeLongRangeCorrections()))
        monte_carlo.set(fst.MakeThermoParams(fst.args({"beta": str(1./temperature),
                                                       "chemical_potential": "1."})))
        monte_carlo.set(fst.MakeMetropolis())
        monte_carlo.add(fst.MakeTrialTranslate(fst.args({"weight": "1.", "tunable_param": "2."})))
        monte_carlo.add(fst.MakeTrialAdd(fst.args({"particle_type": "0"})))
        monte_carlo.run(fst.MakeRun(fst.args({"until_num_particles": str(num_particles)})))
        monte_carlo.run(fst.MakeRemoveTrial(fst.args({"name": "TrialAdd"})))
        monte_carlo.add(fst.MakeLogAndMovie(fst.args({
            "steps_per" : str(steps_per),
            "file_name": "movie",
            "clear_file": "true"})))
        monte_carlo.add(fst.MakeCheckEnergyAndTune(fst.args({"steps_per" : str(steps_per)})))

        # equilibrate
        monte_carlo.attempt(int(1e6))

        # compute average using this script
        energy = fst.MakeAccumulator()

        # production
        for _ in range(int(1e8)):
            monte_carlo.attempt()
            energy.accumulate(monte_carlo.criteria().current_energy())
        
        print('average energy', energy.average())
        for op in range(energy.num_block_operations()):
            print(op, energy.block_stdev(op), energy.block_std_of_std(op))

        #self.assertAlmostEqual(energy.block_stdev(13)/num_particles, 0.0012, delta=0.0004)
        #self.assertAlmostEqual(energy.average()/num_particles, -4.4190, delta=3*energy.block_stdev(13)/num_particles)
       

If the test passes, the energy is within the tolerance of the SRSW value and the two ensemble average methods agreed.

In [2]:
%time  # Note: any line starting with % is only to be used with ipynb
unittest.main(argv=[''], verbosity=2, exit=False)

test_srsw (__main__.TestMonteCarlo4Block) ... 

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 15 µs
average energy -576.5408969939979
0 0.0014677554888216202 2.3057219522546113e-05
1 0.002074833423781676 3.876090615096549e-05
2 0.0029323809037425036 6.514611200974192e-05
3 0.004142167773492622 0.00010943411761494251
4 0.005844603302097843 0.0001836267450924812
5 0.008228670028531814 0.00030744261495308347
6 0.011535650706683156 0.0005125425117659157
7 0.01604201498283397 0.000847665877219287
8 0.021989291856637293 0.0013817730699711033
9 0.029439841014460337 0.0022003232519833657
10 0.03812126183550771 0.0033892288793152684
11 0.04744740126815196 0.005020401506669375
12 0.056964589516646714 0.007196894309296476
13 0.06585558550815984 0.01003017124000322
14 0.07391824998954556 0.013665579361543532
15 0.07946704584674782 0.018029748815872975
16 0.08342377888053773 0.02395333791808359
17 0.0864155468760853 0.030148152258521868
18 0.08571832088343663 0.03296743627085486
19 0.0842442920619326 0.03772153330032034
20 0.0781429587

ok

----------------------------------------------------------------------
Ran 1 test in 1778.262s

OK


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!